In [1]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from util.ZigZag import ZigZag
from util.BuildData import BuildData4
from util.CalcReturns import CalcReturns

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_csv('EURUSD1.csv', header=0)
train = np.array(df.Close)

In [11]:
Z = ZigZag(train, 0.001)
R = CalcReturns(train)
S = BuildData4(Z, R, 241)

#for i in range(len(S["data"])):
#    print "%i: %s %s" % (i, S["data"][i], S["label"][i])

t_count = 1024
print(S['data'].shape)

(13962, 240)


In [12]:
#clf = svm.SVC(kernel="poly", C=0.0001, degree=6, coef0=0.0001)
clf = RandomForestClassifier(n_estimators = 100, criterion='entropy') # 79%
#clf = ExtraTreesClassifier(n_estimators = 10, oob_score = True, bootstrap = True)
#clf = ExtraTreesClassifier(n_estimators = 100)
#clf = DecisionTreeClassifier()
#clf = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 100) # 71%
#clf = GradientBoostingClassifier(n_estimators = 100, learning_rate=1.1) # 81%
#clf = BaggingClassifier() #79 %
#clf = KNeighborsClassifier(n_neighbors=100, algorithm='brute') # 37%
#clf = BaggingClassifier(base_estimator = KNeighborsClassifier(n_neighbors=100)) # 36%
#clf = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1) # 81%

clf.fit(S['data'][:-t_count], S['label'][:-t_count])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [13]:
correct_count  = 0.0

test_buy  = 0.0
test_sell = 0.0

correct_buy  = 0.0
correct_sell = 0.0

incorrect_buy  = 0.0
incorrect_sell = 0.0

p = clf.predict(S['data'][len(S['data'])-t_count:])
c = S['label'][len(S['data'])-t_count:]

for i in range(0, t_count):
    
    if c[i] == 1:
        test_buy += 1.0;
    elif c[i] == -1:
        test_sell += 1.0;
    
    if p[i] == c[i]:
        correct_count += 1.0
        
        if c[i] == 1:
            correct_buy += 1.0;
        elif c[i] == -1:
            correct_sell += 1.0;

    else:        
        if c[i] == 1:
            incorrect_buy += 1.0;
        elif c[i] == -1:
            incorrect_sell += 1.0;            

print( "Accuracy:  ", (correct_count/t_count)*100.00 )

print( "Test buy:  ", test_buy  )
print( "Test sell: ", test_sell )

print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )


('Accuracy:  ', 66.796875)
('Test buy:  ', 459.0)
('Test sell: ', 565.0)
('Correct buy:  ', 266.0, 'Incorrect buy:  ', 193.0, 'Accuracy: ', 57.9520697167756)
('Correct sell: ', 418.0, 'Incorrect sell: ', 147.0, 'Accuracy: ', 73.98230088495575)


In [10]:
%matplotlib qt

plt.plot(S['data'][7])
#plt.plot(Z['time'], Z['zigzag'], range(0, len(train)), train)